In [18]:
import pandas as pd

# Load file
df = pd.read_csv("PUF.DCAS19502005.DAT", sep="|", header=None, dtype=str)

# ---- Parse Date ----
date_col = 34  # assuming column 34 contains YYYYMMDD format
df["Date"] = pd.to_datetime(df[date_col], format="%Y%m%d", errors="coerce")

# Create Year-Quarter column
df["YearQuarter"] = df["Date"].dt.to_period("Q").astype(str)

# ---- Counting ----

# Count H and NH in col 42
col_42 = 42
h_counts = (
    df[df[col_42].str.upper() == "H"]
    .groupby("YearQuarter")
    .size()
    .rename("H")
)
nh_counts = (
    df[df[col_42].str.upper() == "NH"]
    .groupby("YearQuarter")
    .size()
    .rename("NH")
)

# Count "KILLED IN ACTION" in col 44
kia_counts = (
    df[df[44].str.contains("KILLED IN ACTION", case=False, na=False)]
    .groupby("YearQuarter")
    .size()
    .rename("KIA")
)

# Count "HOSTILE" in col 25
hostile_counts = (
    df[df[25].str.contains("HOSTILE ACTION", case=False, na=False)]
    .groupby("YearQuarter")
    .size()
    .rename("HOSTILE ACTION")
)

# ---- Merge all counts ----
counts = pd.concat([h_counts, nh_counts, kia_counts, hostile_counts], axis=1).fillna(0).astype(int).reset_index()

# Save to CSV
counts.to_csv("us_counts_by_quarter.csv", index=False)
print("Saved us_counts_by_quarter.csv")
print(counts.head(10))


Saved us_counts_by_quarter.csv
  YearQuarter     H   NH   KIA  HOSTILE ACTION
0      1950Q2    31    0    22               0
1      1950Q3  8038  113  6273               0
2      1950Q4  6282  184  2501               0
3      1951Q1  3688  204  2558               0
4      1951Q2  3179  258  2433               0
5      1951Q3  2268  277  1935               0
6      1951Q4  2445  223  2022               0
7      1952Q1   582  232   386               0
8      1952Q2   954  243   742               0
9      1952Q3  1558  287  1182               0


In [17]:
print(df.shape)       # rows, cols
print(df.columns)     # see actual column names
# Look at first few rows to spot which column has the "KILLED IN ACTION" text
df.head()

# Normalize the strings for comparison
for col in df.columns:
    # Check if 'KILLED IN ACTION' appears anywhere in this column (case-insensitive)
    if df[col].astype(str).str.contains("KILLED IN ACTION").any():
        print(f"Possible match in column {col}")

# Normalize the strings for comparison
for col in df.columns:
    # Check if 'HOSTILE' appears anywhere in this column (case-insensitive)
    if df[col].astype(str).str.upper().str.contains("HOSTILE").any():
        print(f"Column {col} contains 'HOSTILE ACtion'")


# Normalize the strings for comparison
for col in df.columns:
    # Check if 'NH' appears anywhere in this column (case-insensitive)
    if df[col].astype(str).str.upper().str.contains("NH").any():
        print(f"Column {col} contains 'NH'")

(135795, 57)
Index([            0,             1,             2,             3,
                   4,             5,             6,             7,
                   8,             9,            10,            11,
                  12,            13,            14,            15,
                  16,            17,            18,            19,
                  20,            21,            22,            23,
                  24,            25,            26,            27,
                  28,            29,            30,            31,
                  32,            33,            34,            35,
                  36,            37,            38,            39,
                  40,            41,            42,            43,
                  44,            45,            46,            47,
                  48,            49,            50,            51,
                  52,            53,            54,        'Date',
       'YearQuarter'],
      dtype='object')
Poss

In [49]:
import pandas as pd

file_path = "KIAs 2005 onward maybe2.csv"

# Read CSV without multi-index headers
df = pd.read_csv(file_path)

# Dates are in even-numbered columns, Deaths in odd-numbered columns
date_cols = df.columns[::2]
death_cols = df.columns[1::2]

# Flatten into long format
long_rows = []
for i in range(len(date_cols)):
    date_series = df[date_cols[i]]
    death_series = df[death_cols[i]]

    for date, deaths in zip(date_series, death_series):
        if pd.notna(date) and pd.notna(deaths):
            long_rows.append((date, int(deaths)))  # ensure numeric

long_df = pd.DataFrame(long_rows, columns=["Date", "Deaths"])

# Group by date and sum deaths
final_df = long_df.groupby("Date", as_index=False)["Deaths"].sum()

# Sort by date
final_df = final_df.sort_values("Date")

print(final_df)


         Date  Deaths
0    01/01/02       1
1    01/01/04      39
2    01/01/05      53
3    01/01/06      43
4    01/01/07      78
..        ...     ...
184  12/01/14       3
185  12/01/15       6
186  12/01/16       1
187  12/01/18       1
188  12/01/19       1

[189 rows x 2 columns]


In [50]:
# Convert to datetime
final_df["Date"] = pd.to_datetime(final_df["Date"], format="%m/%d/%y", errors="coerce")
final_df = final_df.dropna(subset=["Date"])

# Extract year and quarter
final_df["Year"] = final_df["Date"].dt.year
final_df["Quarter"] = ((final_df["Date"].dt.month - 1) // 3) + 1

# Group by year and quarter
quarterly_totals = (
    final_df.groupby(["Year", "Quarter"], as_index=False)["Deaths"].sum()
    .sort_values(["Year", "Quarter"])
)
quarterly_totals["YearQuarter"] = quarterly_totals["Year"].astype(str) + " Q" + quarterly_totals["Quarter"].astype(str)
quarterly_totals = quarterly_totals[["YearQuarter", "Deaths"]]

print(quarterly_totals)

quarterly_totals.to_csv("missing_deaths_by_year_quarter.csv", index=False)

   YearQuarter  Deaths
0      1994 Q4       4
1      2001 Q4       3
2      2002 Q1       9
3      2002 Q2       5
4      2002 Q3       1
..         ...     ...
71     2019 Q3       8
72     2019 Q4       1
73     2020 Q1       8
74     2021 Q3      13
75     2024 Q1       3

[76 rows x 2 columns]


In [55]:
import pandas as pd

file_path = "Final_Raw_KIA.csv"

# Read CSV without multi-index headers
organized_df = pd.read_csv(file_path)

# Dates are in even-numbered columns, Deaths in odd-numbered columns
organized_date_cols = organized_df.columns[::2]
organized_death_cols = organized_df.columns[1::2]

# Flatten into long format
organized_long_rows = []
for i in range(len(organized_date_cols)):
    organized_date_series = organized_df[organized_date_cols[i]]
    organized_death_series = organized_df[organized_death_cols[i]]

    for YearQuarter, KIA in zip(organized_date_series, organized_death_series):
        if pd.notna(YearQuarter) and pd.notna(KIA):
            organized_long_rows.append((YearQuarter, int(KIA)))  # ensure numeric

organized_long_df = pd.DataFrame(organized_long_rows, columns=["YearQuarter", "KIA"])

# Group by date and sum deaths
organized_final_df = organized_long_df.groupby("YearQuarter", as_index=False)["KIA"].sum()

# Sort by date
organized_final_df = organized_final_df.sort_values("YearQuarter")

print(organized_final_df)
organized_final_df.to_csv("Final_KIA_by_year_quarter.csv", index=False)

    YearQuarter   KIA
0        1950Q2    22
1        1950Q3  6273
2        1950Q4  2501
3        1951Q1  2558
4        1951Q2  2433
..          ...   ...
194     2019 Q3     8
195     2019 Q4     1
196     2020 Q1     8
197     2021 Q3    13
198     2024 Q1     3

[199 rows x 2 columns]
